# Rag Based

In [ ]:
!pip install sentence_transformers
!pip -q install langchain_community tiktoken chromadb pypdf transformers langchain
!pip -q install accelerate bitsandbytes sentencepiece Xformers
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 31.5 MB/s eta 0:00:

In [ ]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

In [ ]:
# Download From Link(Stock Market Data)
!wget https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip -O stock_market_june_2023.zip
# Unzip the data
!unzip stock_market_june_2023.zip

--2024-11-04 08:35:54--  https://github.com/Shafi2016/Youtube/raw/main/stock_market_june_2023.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip [following]
--2024-11-04 08:35:55--  https://raw.githubusercontent.com/Shafi2016/Youtube/main/stock_market_june_2023.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10638 (10K) [application/zip]
Saving to: ‘stock_market_june_2023.zip’

stock_market_june_2 100%[===================>]  10.39K  --.-KB/s    in 0s      

2024-11-04 08:35:55 (86.7 MB/s) - ‘stock_market_june_2023.zip’ saved [10638/10638

In [ ]:

# Initializes a DirectoryLoader to load text files from the specified directory.
loader = DirectoryLoader('./stock_market_june_2023/', glob="./*.txt", loader_cls=TextLoader)

# Loads all the documents present in the directory.
documents = loader.load()

In [ ]:
documents

[Document(metadata={'source': 'stock_market_june_2023/stock_market.txt'}, page_content='The stock market had a strong month in May, especially for those who read our previous monthly stock recommendation. At The Markets Watch, our journalists and analysts have compiled a list of monthly stock opportunities for buy-and-hold investors and traders looking to buy great companies at an even better price.\nLooking at some of our stock suggestions from the previous month, the evidence of our team’s expertise becomes compelling. In May:\n\nPalantir soared by over 74%\nNvidia witnessed a growth of over 34%\nCrowdstrike Holdings increased by over 30%\nShopify saw a rise of over 19%\nOur insights and articles about these stocks collectively yielded a remarkable 57% return last month for our readers who took action on our recommendations.\n\nInstitutional ETFs assembled by Goldman Sachs and Vanguard rely on growth stocks to drive their price in all market conditions, and your portfolio is no diffe

In [ ]:
# Initializes a RecursiveCharacterTextSplitter with a specified chunk size and overlap.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# Splits the loaded documents into chunks of text using the defined text_splitter.
texts = text_splitter.split_documents(documents)

In [ ]:
len(texts)

49

In [ ]:
texts[1]

Document(metadata={'source': 'stock_market_june_2023/stock_market.txt'}, page_content='Palantir soared by over 74%\nNvidia witnessed a growth of over 34%\nCrowdstrike Holdings increased by over 30%\nShopify saw a rise of over 19%\nOur insights and articles about these stocks collectively yielded a remarkable 57% return last month for our readers who took action on our recommendations.')

In [ ]:
texts[2]

Document(metadata={'source': 'stock_market_june_2023/stock_market.txt'}, page_content='Institutional ETFs assembled by Goldman Sachs and Vanguard rely on growth stocks to drive their price in all market conditions, and your portfolio is no different. Most of our recommendations are already a staple in these ETFs, but for investors looking to cherry-pick the best stocks, we have assembled this list for you.\n\nHere are 8 growth stocks for you to buy in June 2023 set to leverage emerging industry trends, such as AI, the lithium “gold” rush, blockbuster EV sales, and ESG.')

## falcon-7b-instruct

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import transformers
import torch

model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
# Embedding Model
llm = HuggingFacePipeline(pipeline=pipeline)

# Large Model
model_name = "intfloat/e5-large-v2"


hf = HuggingFaceEmbeddings(model_name=model_name)




## Create the DB

In [ ]:
# Sets the directory where the embeddings will be stored.
persist_directory = 'db'

# Sets the HuggingFaceEmbeddings object as the embedding to use.
embedding = hf

# Uses the Chroma module to convert the texts into embeddings using the specified HuggingFace embeddings.
# The resulting embeddings are stored in the persist_directory.
vectordb = Chroma.from_documents(documents=texts,
                                 embedding=hf,
                                 persist_directory=persist_directory)

In [ ]:
# Persists the generated embeddings database to disk.
vectordb.persist()

# Releases the memory held by the vectordb object by setting it to None.
vectordb = None

 #Re-initializes the Chroma object from the persisted directory with the specified HuggingFace embeddings.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=hf)


In [ ]:


# Initializes a retriever object from the vectordb with a specified number of search results (k=3).
# The vectordb.as_retriever() function is likely converting the Chroma object, which contains vectorized representations of text, into a retriever object.
# The retriever can be used to find the most similar vectors in the database given a query vector.
# The search_kwargs={"k": 3} argument suggests that the retriever will return the top 3 most similar vectors for each query.
retriever = vectordb.as_retriever(search_kwargs={"k": 3})

# create the chain to answer questions
# Initializes a RetrievalQA object with the specified HuggingFacePipeline (llm), retriever, and chain type.
# The return_source_documents parameter set to True means the original source documents will be included in the returned results.
qa_chain = RetrievalQA.from_chain_type(llm= llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)


 # This function processes the response from a Language Model (llm) and prints the result and sources.
def process_llm_response(llm_response):
    # Prints the 'result' field from the response, which likely contains the answer from the language model.
    print(llm_response['result'])

    print('\n\nSources:')
    # Iterates over the 'source_documents' field in the response, which contains the documents where the answer was found.
    for source in llm_response["source_documents"]:
        # Prints the 'source' metadata from each document.
        print(source.metadata['source'])

In [ ]:
query = "Could you please enumerate the companies that have been highlighted for their potential stock growth"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 300, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(






Sources:
stock_market_june_2023/stock_market.txt
stock_market_june_2023/stock_market.txt
stock_market_june_2023/Microsoft Corporation.txt


In [ ]:
query = "How much has Microsoft invested in OpenAI?"
llm_response = qa_chain(query)
llm_response

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 280, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


{'query': 'How much has Microsoft invested in OpenAI?',
 'result': ' Microsoft',
 'source_documents': [Document(page_content='Let’s focus on Microsoft’s artificial intelligence (AI) prospects. As AI propels itself into the mainstream, it’s not surprising to see investors eager to fortify their portfolios with AI stocks. Microsoft’s recent $1 billion investment in OpenAI – the creator of ChatGPT and the AI image generator Dall-E- positions the stock as a prime candidate for investment.', metadata={'source': 'stock_market_june_2023/Microsoft Corporation.txt'}),
  Document(page_content='Microsoft’s strategic move to integrate OpenAI’s products within its platform positions the company as a preferred platform for organizations looking to develop AI models or incorporate them into their business operations. Microsoft’s focus on building AI tools as platforms is a savvy strategic decision that will likely broaden its already considerable enterprise customer base and service suite.', metadata

In [ ]:
query = "What were the reasons behind Shopify's decision to lay off a portion of its workforce"
llm_response = qa_chain(query)
process_llm_response(llm_response)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 240, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(






Sources:
stock_market_june_2023/Shopify.txt
stock_market_june_2023/Shopify.txt
stock_market_june_2023/Shopify.txt
